## 3. 크롤링 - 브랜드/모델 이름&코드 테이블 생성

### 브랜드 코드/이름 테이블

In [292]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as BS
from tqdm import tqdm
import time
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="131.0.6778.70").install()),options=options)

# 브랜드 코드 조회
url = "https://auto.danawa.com/auto/?Work=home"
driver.get(url)
time.sleep(3)
bs3 = BS(driver.page_source)

li_brand = list()
# 국내
for br in bs3.find('div', class_='domestic').findAll('li'):
    # 브랜드 이름
    nm_brand = br.text
    # 브랜드 코드
    cd_brand = br.find('a')['href'].split('=')[-1]
    li_brand.append([cd_brand, nm_brand, 1])
    
# 수입
for br in bs3.find('div', class_='import').findAll('li'):
    # 브랜드 이름
    nm_brand = br.text
    # 브랜드 코드
    cd_brand = br.find('a')['href'].split('=')[-1]
    li_brand.append([cd_brand, nm_brand, 0])

df_brand = pd.DataFrame(li_brand, columns = ['cd_brand','nm_brand','domestic_bit'])


In [294]:
df_brand

,cd_brand,nm_brand,domestic_bit
0,303,현대,1
1,307,기아,1
2,304,제네시스,1
3,312,쉐보레,1
4,321,르노코리아,1
...,...,...,...
66,528,스바루,0
67,518,미쓰비시,0
68,610,북기은상,0
69,583,크라이슬러,0


### 차량모델 코드/이름 테이블

In [316]:
# 브랜드 코드 리스트
li_brand_cd = df_brand.cd_brand.tolist()

li_model = list()
for cd_brand in tqdm(li_brand_cd):
    url = f"https://auto.danawa.com/auto/?Work=brand&Brand={cd_brand}"
    driver.get(url)
    time.sleep(3)
    bs4 = BS(driver.page_source)
    
    if bs4.find('dl', class_='modelBox leftType clearFix') != None:
        
        for mo in bs4.find('dl', class_='modelBox leftType clearFix').findAll('li'):
            # 차량모델 코드
            cd_model = mo['code']
            # 차량모델 이름
            nm_model = mo.text.strip()
            li_model.append([cd_model, nm_model, cd_brand])
            
df_model = pd.DataFrame(li_model, columns = ['cd_model','nm_model','cd_brand'])

100%|█████████████████████████████████████| 71/71 [04:23<00:00,  3.72s/it]


In [317]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cd_model  428 non-null    object
 1   nm_model  428 non-null    object
 2   cd_brand  428 non-null    object
dtypes: object(3)
memory usage: 10.2+ KB


In [318]:
df_model

,cd_model,nm_model,cd_brand
0,4086,캐스퍼,303
1,4671,더 뉴 캐스퍼,303
2,4653,캐스퍼 일렉트릭,303
3,4455,더 뉴 아반떼,303
4,4564,더 뉴 아반떼 N,303
...,...,...,...
423,4583,Rich6 EV,617
424,3487,C31,617
425,3488,C32,617
426,4453,1500 Revolution BEV Concept,565


### DB에 테이블로 저장

In [320]:
# DB 연결
import sqlalchemy
from urllib import parse
user='team2'
password='Encore_team2@'
password = parse.quote_plus(password)
host='192.168.0.95'
#host='222.112.208.67'
port=3306
database='team2'
engine=sqlalchemy.create_engine(f'mysql://{user}:{password}@{host}:{port}/{database}')

df_brand.to_sql('brand_cd', if_exists='replace', con=engine, index=False)
df_model.to_sql('model_cd', if_exists='replace', con=engine, index=False)

428